In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
import json

In [0]:
# ADLS paths
tiers = ["bronze", "silver", "gold"]
adls_paths = {tier: f"abfss://{tier}@adlschinook.dfs.core.windows.net" for tier in tiers}
bronze_path = adls_paths["bronze"]
silver_path = adls_paths["silver"]
gold_path = adls_paths["gold"]
bronze_path

'abfss://bronze@adlschinook.dfs.core.windows.net'

In [0]:
# list the files inside the bronze firectory
bronze_directory= dbutils.fs.ls(f"{bronze_path}/dbo")
# split the filename to just get the tablenames
table_names =[]
for files in bronze_directory:
    table_names.append(files.name.split("/")[0])
table_names

['Album',
 'Artist',
 'Customer',
 'Employee',
 'Genre',
 'Invoice',
 'InvoiceLine',
 'MediaType',
 'Playlist',
 'PlaylistTrack',
 'Track']

In [0]:
# iterate through each file and make transformations if necessary and write ti the silver ADLS
for name in table_names:
  file_path = f"{bronze_path}/dbo/{name}"
  df = spark.read\
    .format("parquet")\
      .load(file_path)
  
  if name == "Employee":
    df= df.withColumn("BirthDate", col("BirthDate").cast("date"))\
      .withColumn("HireDate", col("HireDate").cast("date"))
    
  if name == "Invoice":
    df= df.withColumn("InvoiceDate", col("InvoiceDate").cast("date"))\
      .withColumn("PricePerQuantity", lit(0.99))\
        .withColumn("Quantity", (col("Total")/0.99).cast("integer"))
 
  if name == "Track":
    df= df.withColumn("Seconds", (col("Milliseconds")/1000).cast("integer"))\
      .withColumn("MegaBytes", round(col("Bytes")/1024/1024, 2))
    df= df.drop("Milliseconds", "Bytes")

  try:
    df.write\
      .format("parquet")\
        .mode("overwrite")\
          .save(f"{silver_path}/dbo/{name}")
    print(f"Table {name} written to silver ADLS successfully")
  except Exception as e:
    print(f"Table {name} failed to write to silver ADLS: {e}")

Table Album written to silver ADLS successfully
Table Artist written to silver ADLS successfully
Table Customer written to silver ADLS successfully
Table Employee written to silver ADLS successfully
Table Genre written to silver ADLS successfully
Table Invoice written to silver ADLS successfully
Table InvoiceLine written to silver ADLS successfully
Table MediaType written to silver ADLS successfully
Table Playlist written to silver ADLS successfully
Table PlaylistTrack written to silver ADLS successfully
Table Track written to silver ADLS successfully


In [0]:
# pass the output parameters
output_params = {
    "bronze_path": bronze_path,
    "silver_path": silver_path,
    "gold_path": gold_path
} 
dbutils.notebook.exit(json.dumps(output_params))